In [1]:
import os
from getpass import getpass

from dotenv import load_dotenv



In [2]:
import pstuts_rag

In [3]:
%load_ext autoreload
%autoreload 2


In [4]:

load_dotenv()

def set_api_key_if_not_present(key_name, prompt_message=""):
    if len(prompt_message) == 0:
        prompt_message=key_name
    if key_name not in os.environ or not os.environ[key_name]:
        os.environ[key_name] = getpass.getpass(prompt_message)

set_api_key_if_not_present("OPENAI_API_KEY")
set_api_key_if_not_present("TAVILY_API_KEY")

In [5]:
from dataclasses import dataclass
@dataclass
class ApplicationParameters:
    filename = "data/test.json"
    embedding_model = "text-embedding-3-small"
    tool_calling_model = "gpt-4.1-mini"
    n_context_docs = 2

params = ApplicationParameters()

In [6]:
import functools
import operator
from typing import Annotated, List, TypedDict

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI

class PsTutsTeamState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: str

In [7]:
llm_tool_calling = ChatOpenAI(model=params.tool_calling_model,temperature=0)

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

adobe_help_search = TavilySearchResults(max_results=5,include_domains=["helpx.adobe.com"])

In [9]:
from pstuts_rag.agents import create_agent, agent_node

adobe_help_agent = create_agent(
    llm_tool_calling,
    [adobe_help_search],
    "You are a research assistant who can search"
    "for Adobe Photoshop help topics using the tavily search engine."
    "Users may provide you with partial questions - try your best to determine their intent."
    "If sending a request to search, craft your query so that it enhances user's ability"
    "to receive a helpful answer.",
)
adobe_help_node = functools.partial(agent_node, agent=adobe_help_agent, name="AdobeHelp")

In [10]:
retval = adobe_help_agent.invoke({"question":"What are layers?","messages":[],"team_members":[]})

In [11]:
retval

{'question': 'What are layers?',
 'messages': [],
 'team_members': [],
 'output': 'You can find comprehensive Adobe Photoshop help topics and user guides on the official Adobe website. These cover a wide range of subjects including color management, web and screen design, video and animation, nondestructive editing, layers and groups, masks, Smart Filters, blending modes, and more.\n\nHere are some useful links to explore:\n- Common questions about Photoshop on the web: https://helpx.adobe.com/photoshop/using/photoshop-web-faq.html\n- Photoshop User Guide: https://helpx.adobe.com/photoshop/user-guide.html\n- Photoshop tools, options, and task bars: https://helpx.adobe.com/photoshop/using/using-tools.html\n- View all Adobe Photoshop tutorials: https://helpx.adobe.com/in/photoshop/view-all-tutorials.filter-bar.html#!topic-title\n\nThese resources provide detailed instructions and tutorials to help you with various Photoshop features and tasks.'}

# Data Preparation

First, we will read in the transcripts of the videos and convert them to Documents
with appropriate metadata.

In [12]:
from ast import Dict
import json

from pstuts_rag.loader import load_json_files
filename = ["../data/test.json"]
from typing import List, Dict, Any
data:List[Dict[str,Any]] = await load_json_files(filename)


## R - retrieval

Let's hit it with a semantic chunker.

In [13]:
from pstuts_rag.datastore import DatastoreManager
from qdrant_client import QdrantClient

client = QdrantClient(path=":memory:")

retriever_factory = DatastoreManager(qdrant_client=client,name="local_test")
if retriever_factory.count_docs() == 0:
    await retriever_factory.populate_database(raw_docs=data)

## Generation

We will use a 4.1-nano to generate answers.

In [24]:
from json import tool
from pstuts_rag.rag import RAGChainFactory
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

rag_factory = RAGChainFactory(retriever=retriever_factory.get_retriever())
get_videos = rag_factory.get_rag_chain(llm=ChatOpenAI(model=params.tool_calling_model))

@tool
def get_videos_call(
    query: Annotated[str, "Query to pose to Photoshop training video transcript archive"]
    ):
  """Extract information from Photoshop training video archive."""
  return get_videos.invoke({"question" : query})

get_videos_agent = create_agent(
  llm_tool_calling,
  [get_videos_call],
  """You are an expert trainer in Adobe Photoshop who
  has an archive of training videos at her disposal.
  You can request transcripts of those videos and then summarize and shape 
  them to provide helpful answers."""
)

get_videos_node = functools.partial(agent_node, 
                                    agent=get_videos_agent,
                                    name="VideoArchiveSearch")


In [25]:
retval = get_videos_call.invoke("What are layers?")
retval

AIMessage(content='Layers in Photoshop are like separate flat pieces of glass stacked on top of each other, where each layer contains its own separate content. This allows you to work on different parts of an image independently. For example, you can toggle the visibility of each layer on or off to see what content it holds. Transparent areas in a layer let you see through to the layers below. This stacking and independence are the biggest benefits of using layers—you can edit one part of an image without affecting others. You can see all layers and work with them in the Layers panel. (See 0:28–2:00 and 1:25–2:32 for details) 🎨🖼️\n**REFERENCES**\n[\n  {\n    "title": "Understand layers",\n    "source": "https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4",\n    "start": 0.47,\n    "stop": 62.14\n  },\n  {\n    "title": "Understand layers",\n    "source": 

In [26]:
val = await get_videos.ainvoke({"question":"What are layers"})

In [51]:
from pstuts_rag.agents import create_team_supervisor
supervisor_agent = create_team_supervisor(
    llm_tool_calling,
    """You are the Supervisor for an agentic RAG system. Your job is to interpret the user’s request, extract the core research topic, and decide which research-focused worker to invoke next. Reply only with the next worker and the subject to research, or FINISH when the workflow is complete.

Workers
• VideoArchiveSearch – retrieves videos related to the query  
• AdobeHelp         – searches Adobe’s documentation and training resources  

Routing Rules
1. Topic Extraction  
   • Read the user’s request and identify a concise research topic (e.g. “Photoshop timeline keyframes”).

2. Primary Preference  
   • First invoke VideoArchiveSearch with that topic.  
   • If VideoArchiveSearch returns “I don’t know” or “no results,” fall back to AdobeHelp.

3. AdobeHelp Behavior  
   • When routing to AdobeHelp, always also check for related training videos in Adobe’s library.

4. Research-Only  
   • Only invoke workers that perform research tasks.

5. Completion  
   • When neither worked can provide value, go to FINISH. If AdobeHelp
   expands the list of topics, make sure to attempt to search for them with the
   VideoArchiveSearch.

Response Format
<WorkerName>: <Research Topic>

Example:
VideoArchiveSearch: exporting vector layers from After Effects

And, once there’s no further research needed:
FINISH
""",
    ["VideoArchiveSearch", "AdobeHelp"],
)



## Graph Creation

In [52]:
from langgraph.graph import END, StateGraph

adobe_help_graph = StateGraph(PsTutsTeamState)

adobe_help_graph.add_node("VideoArchiveSearch", get_videos_node)
adobe_help_graph.add_node("AdobeHelp", adobe_help_node)
adobe_help_graph.add_node("supervisor", supervisor_agent)

edges = [
    ["VideoArchiveSearch","supervisor"],
    ["AdobeHelp","supervisor"],
]

[adobe_help_graph.add_edge(*p) for p in edges]

adobe_help_graph.add_conditional_edges(
    "supervisor",
    lambda x:x["next"],
    {"VideoArchiveSearch":"VideoArchiveSearch",
    "AdobeHelp":"AdobeHelp",    
    "FINISH": END},
)
adobe_help_graph.set_entry_point("supervisor")
adobe_help_compiled = adobe_help_graph.compile()

In [53]:
adobe_help_graph.set_entry_point("supervisor")
compiled_research_graph = adobe_help_graph.compile()

Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.


In [54]:
import nest_asyncio
nest_asyncio.apply()

In [55]:
from langchain_core.runnables.graph_ascii import draw_ascii

graph_data = compiled_research_graph.get_graph()
print(graph_data.draw_ascii())

                            +-----------+                             
                            | __start__ |                             
                            +-----------+                             
                                  *                                   
                                  *                                   
                                  *                                   
                            +------------+                            
                            | supervisor |                            
                       *****+------------+.....                       
                   ****            *           ....                   
              *****                *               .....              
           ***                     *                    ...           
+-----------+           +--------------------+           +---------+  
| AdobeHelp |           | VideoArchiveSearch |           | __end__ |  
+-----

In [56]:
def enter_chain(message: str):
    results = {
        "messages": [HumanMessage(content=message)],
        "team_members": ["VideoArchiveSearch", "AdobeHelp"],
    }
    return results

research_chain = enter_chain | compiled_research_graph

def demo_research_chain(query:str):
    
    for s in research_chain.stream(
        query, 
        {"recursion_limit": 20}
    ):
        if "__end__" not in s:
            if 'supervisor' not in s.keys():
                for response in s.values():
                    for msg in response['messages']:
                        msg.pretty_print()
            else:
                print(s)
            print("---")
            


In [58]:
demo_research_chain("What are layers?")

{'supervisor': {'next': 'VideoArchiveSearch'}}
---
================================ Human Message =================================
Name: VideoArchiveSearch

Layers in Photoshop are like separate flat panes of glass stacked on top of each other, with each layer containing separate pieces of content. You can think of them as the building blocks of any image. In the Layers panel, you can see all the layers in your image, each with an Eye icon to toggle its visibility on or off, allowing you to see or hide that layer's content.

A single layer can have transparent pixels, shown as a gray and white checkerboard pattern, which lets you see through to the layers beneath it. You can also hold down the Option key (Mac) or ALT key (PC) and click the Eye icon next to a layer to hide all other layers except the one you clicked on, focusing only on that layer. Doing the same action again will bring all other layers back into view.

The main benefit of using layers is that you can edit parts of you